In [318]:
import pandas as pd

In [319]:
df1 = pd.read_excel("C:/박규영/경남공모전/본선자료/1.병원정보서비스 2022.12.xlsx")
df3 = pd.read_csv("C:/박규영/경남공모전/변수 종합(full 버전).csv")

로딩 오래 걸리니까 중간 저장해놓는걸루

In [320]:
col = ['요양기관명', '종별코드명', '시군구코드명',
       '읍면동', '우편번호', '주소', '총의사수', '의과일반의 인원수','좌표(X)', '좌표(Y)']
df1 = df1[col] # 필요컬럼만 추출

df1 = df1[df1['시군구코드명'].str.contains('창원')] # 창원시만 추출 

df1 = df1[(df1['종별코드명'] != '정신병원') & (df1['종별코드명'] != '요양병원') & (df1['종별코드명'] != '조산원')] # 필요병원만 추출
df1 = df1[~df1['요양기관명'].str.contains('산부')] # 종별코드명에서 의원으로 나타났지만 산부인과인 의원 제외
# df1 = df1[~df1['요양기관명'].str.contains('피부')] # 종별코드명에서 의원으로 나타났지만 피부과인 의원 제외

df1['읍면동'] = df1['주소'].str.extract(r'\((.*?)\)') # 읍면동 추출1
df1['읍면동'] = df1['읍면동'].str.split(',').str[0] # 읍면동 추출2

In [72]:
df1['읍면동'].unique()

array(['합성동', '중앙동3가', '중앙동', '이동', '청아병원', '월남동2가', '명서동', '성주동', '사림동',
       '석전동', '상남동', '가포동', '팔용동', '서상동', '중앙동2가', '삼계웰빙프라자빌딩', '창포동3가',
       '자은동', '용원동', '남성동', '소계동', '양덕동', nan, '신포동2가', '장천동', '동성동',
       '중동', '남양동', '오동동', '봉곡동', '신촌동', '반지동', '반림동', '성산동', '가음정동',
       '가음동', '산호동', '서성동', '신월동', '마산운전면허시험장', '회원동', '용호동', '고운맘산부인과의원',
       '월영동', '내서읍', '석동', '여좌동', '실로암빌딩', '해운동', '진동면', '회성동', '메디팜중앙약국',
       '대방동', '대원동', '두월동1가', '남문동', '내동', '사파동', '풍호동', '하나로마트', '도계동',
       '덕산빌딩2층', '귀곡동', '701호일부', '포인포', '튼튼한약국', '교방동', '경화동', '2층 203호',
       '월남동5가', '월남동4가', '소답동', '대산면', '송학동', '충무동', '봉암동', '',
       '코오롱그랜드쇼핑', '하이존빌딩', '현동', '수성동', '구암동', '북면', '신성네스트빌', '자산동',
       '덕산동', '안민동', '동정동', '눈나라안경', '봉림동', '새마을금고', '화진빌딩', '303',
       '재성빌딩 204호', '마천동', '남해횟집', '2층', '부림동', '중앙상가', '2', '가나약국',
       '장군동3가', '남산동', '지상휘신경외과', '북동', '열린약국', '창동', '한국한센복지협회', '중앙동1가',
       '진동수산업협동조합', '완월동', '중성동', '파워콤', '코아상가 401호', '삼계대동이미지2차아파트'

In [151]:
len(df1)

1261

- 읍면동이 ㅇㅇ동 형태가 아닌 데이터만 따로 부름
- 아마 동이 포함되어 있지 않은 애들 중에는 '내서읍' '동읍'이런애들이라 그럼

### 동이 포함된 데이터 vs 동이 포함되지 않은 데이터 구분
- 동이 포함된 데이터: 동 이름이 아니라 102동 이런애들 있으니 추가 전처리 필요
- 동이 포함되지 않은 데이터: 내서읍, 동읍 이런애들이니까 데이터 확인 후 전처리

#### 1. 동이 포함되지 않은 데이터

In [321]:
df1['읍면동'] = df1['읍면동'].astype(str) # str로 바꿔야 밑에께 돌아감

error_df1 = df1[~df1['읍면동'].str.contains('동')]
error_df1['읍면동'] = error_df1['주소'].str.split(' ').str[3]

C:\Users\koaro\AppData\Local\Temp\ipykernel_18696\4172095476.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  error_df1['읍면동'] = error_df1['주소'].str.split(' ').str[3]


In [153]:
error_df1.isnull().sum()

요양기관명        0
종별코드명        0
시군구코드명       0
읍면동          0
우편번호         0
주소           0
총의사수         0
의과일반의 인원수    0
좌표(X)        0
좌표(Y)        0
dtype: int64

In [154]:
error_df1['읍면동'].unique()

array(['내서읍', '진동면', '북면', '가양로124번길', '진해대로776번길', '동읍', '대산면', '진해대로',
       '구산면', '3·15대로', '진북면', '진전면'], dtype=object)

- 결측값은 없으나 unique()를 보면 이상한 애들이 있음
- 이상한 애들만 따로 불러서 확인 작업 필요

In [322]:
error_list = ['가양로124번길', '진해대로776번길','중동중앙로','진해대로', '3·15대로']
error_df1[error_df1['읍면동'].str.contains('|'.join(error_list))]

,요양기관명,종별코드명,시군구코드명,읍면동,우편번호,주소,총의사수,의과일반의 인원수,좌표(X),좌표(Y)
5664,구교준이비인후과의원,의원,창원성산구,가양로124번길,51465,경상남도 창원시 성산구 가양로124번길 17-12 301호(실로암빌딩) (대방동),1,0,128.707867,35.207807
9696,더나은정형외과의원,의원,창원진해구,진해대로776번길,51658,경상남도 창원시 진해구 진해대로776번길 28 진해농협종합회관(하나로마트) B동 2...,1,0,128.698087,35.156625
13815,바른피부과의원,의원,창원진해구,진해대로776번길,51658,경상남도 창원시 진해구 진해대로776번길 28 진해농협종합회관(하나로마트) 석동동 ...,1,0,128.698087,35.156625
16968,상쾌한이비인후과의원,의원,창원진해구,진해대로,51642,"경상남도 창원시 진해구 진해대로 999 202호, 207호(하이존빌딩) (자은동)",1,0,128.707814,35.140264
25838,예일이비인후과의원,의원,창원마산회원구,3·15대로,51304,"경상남도 창원시 마산회원구 3·15대로 645 예일메디컬센터 2,8,9층 3층(30...",5,0,128.576436,35.230974
31900,조준형내과의원,의원,창원진해구,진해대로776번길,51658,경상남도 창원시 진해구 진해대로776번길 28 진해농협종합회관(하나로마트) B동 2...,1,0,128.698087,35.156625
49076,안종우통합치과의원,치과의원,창원진해구,진해대로776번길,51658,경상남도 창원시 진해구 진해대로776번길 28 진해농협종합회관(하나로마트) 1층 (석동),1,0,128.698087,35.156625
68493,서울부부한의원,한의원,창원진해구,진해대로776번길,51658,경상남도 창원시 진해구 진해대로776번길 28 진해농협종합회관(하나로마트) 1층 (석동),2,0,128.698087,35.156625


주소가 끝까지 안나와있는 애들의 경우 네이버 검색, 보이는 애들의 경우 그대로 처리

In [323]:
error_df1.loc[5664, '읍면동'] = '대방동'
error_df1.loc[9696, '읍면동'] = '석동'
error_df1.loc[13815, '읍면동'] = '석동'
error_df1.loc[16968, '읍면동'] = '자은동'
error_df1.loc[25838, '읍면동'] = '석전동'
error_df1.loc[31900, '읍면동'] = '석동'
error_df1.loc[49076, '읍면동'] = '석동'
error_df1.loc[68493, '읍면동'] = '석동'

#### 2. 동이 포함된 데이터

In [324]:
none_error_df1 = df1[df1['읍면동'].str.contains('동')]

In [163]:
none_error_df1['읍면동'].unique()

array(['합성동', '중앙동3가', '중앙동', '이동', '월남동2가', '명서동', '성주동', '사림동', '석전동',
       '상남동', '가포동', '팔용동', '서상동', '중앙동2가', '창포동3가', '자은동', '용원동', '남성동',
       '소계동', '양덕동', '신포동2가', '장천동', '동성동', '중동', '남양동', '오동동', '봉곡동',
       '신촌동', '반지동', '반림동', '성산동', '가음정동', '가음동', '산호동', '서성동', '신월동',
       '회원동', '용호동', '월영동', '석동', '여좌동', '해운동', '진동면', '회성동', '대방동',
       '대원동', '두월동1가', '남문동', '내동', '사파동', '풍호동', '도계동', '귀곡동', '교방동',
       '경화동', '월남동5가', '소답동', '송학동', '충무동', '봉암동', '현동', '수성동', '구암동',
       '월남동4가', '자산동', '덕산동', '안민동', '봉림동', '마천동', '부림동', '장군동3가', '남산동',
       '북동', '동정동', '창동', '중앙동1가', '진동수산업협동조합', '완월동', '중성동',
       '삼계대동이미지2차아파트', '동읍빌딩', '교원동', '961-2 A동 1층', '청안동', '월남동3가', '동읍',
       '화천동', '안골동', '태평동', '장군동5가', '월남동1가', '장군동4가', '창선동', '근화동'],
      dtype=object)

In [325]:
non_list = ['진동수산업협동조합','삼계대동이미지2차아파트','동읍빌딩','961-2 A동 1층']
none_error_df1[none_error_df1['읍면동'].str.contains('|'.join(non_list))]

,요양기관명,종별코드명,시군구코드명,읍면동,우편번호,주소,총의사수,의과일반의 인원수,좌표(X),좌표(Y)
37819,혜승의원,의원,창원마산합포구,진동수산업협동조합,51792,경상남도 창원시 마산합포구 진동면 해양관광로 20 (진동수산업협동조합),1,1,128.486745,35.113100
41653,대동치과의원,치과의원,창원마산회원구,삼계대동이미지2차아파트,51240,경상남도 창원시 마산회원구 내서읍 광려로 37 (삼계대동이미지2차아파트),1,0,128.502832,35.223295
42584,류치과의원,치과의원,창원의창구,동읍빌딩,51132,경상남도 창원시 의창구 동읍 의창대로 854 2층 201호 (동읍빌딩),1,0,128.684600,35.276840
50733,예담치과의원,치과의원,창원의창구,961-2 A동 1층,51103,경상남도 창원시 의창구 북면 천주로 1107 (961-2 A동 1층),1,0,128.606828,35.349275


In [326]:
none_error_df1.loc[37819, '읍면동'] = '진동면' 
none_error_df1.loc[41653, '읍면동'] = '내서읍 ' 
none_error_df1.loc[42584, '읍면동'] = '동읍 ' 
none_error_df1.loc[50733, '읍면동'] = '북면  ' 

#### 3. 두 데이터 병함

In [168]:
len(error_df1) + len(none_error_df1) # 길이 확인

1261

In [327]:
df1 = pd.concat([none_error_df1, error_df1], ignore_index=True)

### 읍면동명을 행정동명으로 바꾸기

In [328]:
eup_list = set(df1['읍면동'].unique())
hang_list = set(df3['행정동'].unique())

In [329]:
common_names = eup_list.intersection(hang_list)

In [288]:
print(sorted(common_names))

['가음정동', '가포동', '경화동', '교방동', '구산면', '내서읍', '대산면', '덕산동', '동읍', '봉림동', '봉암동', '북면', '사파동', '산호동', '상남동', '석동', '석전동', '성주동', '여좌동', '오동동', '완월동', '월영동', '이동', '자산동', '자은동', '중앙동', '진동면', '진북면', '진전면', '충무동', '풍호동', '현동', '회성동']


- 읍면동 = 행정동인 데이터는 그대로 냅두기
- 읍면동 != 행정동인 데이터 추가 전처리

#### 1. 읍면동 = 행정동인 데이터

In [330]:
common_df1 = df1[df1['읍면동'].isin(common_names)]
common_df1.head(3)

,요양기관명,종별코드명,시군구코드명,읍면동,우편번호,주소,총의사수,의과일반의 인원수,좌표(X),좌표(Y)
2,근로복지공단 창원병원,종합병원,창원성산구,중앙동,51524,경상남도 창원시 성산구 창원대로 721 (중앙동),30,2,128.671751,35.218754
3,연세에스병원,종합병원,창원진해구,이동,51659,경상남도 창원시 진해구 해원로32번길 13 (이동),23,1,128.698149,35.154933
7,창원경상국립대학교병원,종합병원,창원성산구,성주동,51472,"경상남도 창원시 성산구 삼정자로 11 (성주동, 창원경상대학교병원)",177,0,128.707846,35.199087


#### 2. 읍면동 != 행정동인 데이터

In [331]:
uncommon_df1 = df1[~df1['읍면동'].isin(common_names)]
uncommon_df1.head(3)

,요양기관명,종별코드명,시군구코드명,읍면동,우편번호,주소,총의사수,의과일반의 인원수,좌표(X),좌표(Y)
0,학교법인성균관대학삼성창원병원,상급종합,창원마산회원구,합성동,51353,"경상남도 창원시 마산회원구 팔용로 158-158 (합성동, 삼성창원병원)",292,1,128.592025,35.242877
1,경상남도마산의료원,종합병원,창원마산합포구,중앙동3가,51736,경상남도 창원시 마산합포구 3·15대로 231 (중앙동3가),21,1,128.565597,35.199237
4,의료법인석영의료재단창원제일종합병원,종합병원,창원마산합포구,중앙동3가,51732,경상남도 창원시 마산합포구 3·15대로 238 (중앙동3가),15,0,128.567600,35.199085


In [217]:
uncommon_df1['읍면동'].unique()

array(['합성동', '중앙동3가', '월남동2가', '명서동', '사림동', '팔용동', '서상동', '중앙동2가',
       '창포동3가', '용원동', '남성동', '소계동', '양덕동', '신포동2가', '장천동', '동성동', '중동',
       '남양동', '봉곡동', '신촌동', '반지동', '반림동', '성산동', '가음동', '서성동', '신월동',
       '회원동', '용호동', '해운동', '대방동', '대원동', '두월동1가', '남문동', '내동', '도계동',
       '귀곡동', '월남동5가', '소답동', '송학동', '수성동', '구암동', '월남동4가', '안민동', '마천동',
       '부림동', '장군동3가', '남산동', '북동', '동정동', '창동', '중앙동1가', '중성동', '내서읍 ',
       '동읍 ', '교원동', '북면  ', '청안동', '월남동3가', '화천동', '안골동', '태평동', '장군동5가',
       '월남동1가', '장군동4가', '창선동', '근화동'], dtype=object)

동읍이 잇는데 '동읍 ' 한칸띄어져있어서 추가된듯

#### 3. 읍면동 -> 행정동 변환을 위한 데이터프레임 생성
https://www.changwon.go.kr/cwportal/10671/10672/10679.web 참고

In [332]:
data = {
    '읍면동': ['명서동', '월남동', '합성동', '반지동', '사림동', '팔용동', '소계동', '봉곡동', '남양동', '서상동', '신촌동', '양덕동', '남성동', '동성동'],
    '행정동': ['명곡동', '문화동', '석전동', '반송동', '봉림동', '팔룡동', '팔룡동', '봉림동', '웅동1동', '의창동', '웅남동', '양덕2동', '오동동', '오동동']
}

new_df = pd.DataFrame(data, columns=['읍면동', '행정동'])
new_df.columns = ['읍면동', '행정동']

In [333]:
additional_data = {
    '읍면동': ['마천동', '대흥동', '송학동', '성산동', '교원동', '중성동', '청안동', '창선동', '화천동', '안골동', '귀곡동', '남산동'],
    '행정동': ['웅동1동', '충무동', '충무동', '웅남동', '교방동', '오동동', '웅동2동', '충무동', '충무동', '웅동2동', '웅남동', '사파동']
}

additional_df = pd.DataFrame(additional_data, columns=['읍면동', '행정동'])
new_df = pd.concat([new_df, additional_df], ignore_index=True)
new_df.columns = ['읍면동', '행정동']

In [334]:
additional_data2 = {
    '읍면동': ['용원동', '용호동', '반계동', '소답동', '대원동', '해운동', '대성동', '덕동동', '두척동', '구암동', '회원동', '동서동', '장천동', '도계동'],
    '행정동': ['웅동2동', '용지동', '팔룡동', '의창동', '팔룡동', '월용동', '반월중앙동', '현동', '회성동', '구암2동', '회원2동', '오동동', '풍호동', '명곡동']
}

additional_df2 = pd.DataFrame(additional_data2, columns=['읍면동', '행정동'])
new_df = pd.concat([new_df, additional_df2], ignore_index=True)
new_df.columns = ['읍면동', '행정동']

In [335]:
additional_data3 = {
    '읍면동': ['가음동', '반림동', '동정동', '대방동', '신월동', '안민동', '수성동', '서성동', '신포동', '장군동', '두월동', '부림동', '남문동'],
    '행정동': ['가음정동', '반송동', '의창동', '사파동', '용지동', '성주동', '반월중앙동', '오동동', '오동동', '완월동', '문화동', '오동동', '웅천동']
}

additional_df3 = pd.DataFrame(additional_data3, columns=['읍면동', '행정동'])
new_df = pd.concat([new_df, additional_df3], ignore_index=True)
new_df.columns = ['읍면동', '행정동']

In [336]:
kkk = pd.merge(new_df, uncommon_df1, on='읍면동', how='right')

그럼에도 행정동 없는애들이 있어 별도로 처리...

In [337]:
null = kkk[kkk['행정동'].isnull()] 

In [338]:
mapping_dict = {
    '근화동': '충무동',
    '내동': '중앙동',
    '내서읍 ': '내서읍',
    '동읍 ': '동읍',
    '두월동1가': '문화동',
    '북동': '의창동',
    '북면  ': '북면',
    '신포동2가': '오동동',
    '월남동1가': '문화동',
    '월남동2가': '문화동',
    '월남동3가': '문화동',
    '월남동4가': '문화동',
    '월남동5가': '문화동',
    '장군동': '반월중앙동',
    '장군동3가': '반월중앙동',
    '장군동4가': '반월중앙동',
    '장군동5가': '반월중앙동',
    '중동': '의창동',
    '중앙동1가': '중앙동',
    '중앙동2가': '중앙동',
    '중앙동3가': '중앙동',
    '창동': '오동동',
    '창포동3가': '문화동',
    '태평동': '충무동'
}

# Step 2: Map the '읍면동' column with the '행정동' values using the mapping dictionary
null['행정동'] = null['읍면동'].map(mapping_dict)

C:\Users\koaro\AppData\Local\Temp\ipykernel_18696\2573038334.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  null['행정동'] = null['읍면동'].map(mapping_dict)


In [339]:
null.isnull().sum()

읍면동          0
행정동          0
요양기관명        0
종별코드명        0
시군구코드명       0
우편번호         0
주소           0
총의사수         0
의과일반의 인원수    0
좌표(X)        0
좌표(Y)        0
dtype: int64

### 최종 합체
- common_df1 (행정동=읍면동)
- kkk (행정동 != 읍면동 1차 가공)
- null (행정동 != 읍면동 2차 가공)

In [340]:
common_df1['행정동'] = common_df1['읍면동']

kkk = kkk[~kkk['행정동'].isnull()] 

final = pd.concat([common_df1, kkk], ignore_index=True)

final = pd.concat([final, null], ignore_index=True) # 

final = final[~final['요양기관명'].str.contains('피부')] #  피부과 뺸다 만다?

C:\Users\koaro\AppData\Local\Temp\ipykernel_18696\2248375949.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  common_df1['행정동'] = common_df1['읍면동']


In [341]:
final['행정동'].unique()

array(['중앙동', '이동', '성주동', '석전동', '상남동', '가포동', '자은동', '오동동', '가음정동',
       '산호동', '월영동', '석동', '여좌동', '진동면', '회성동', '사파동', '풍호동', '교방동',
       '경화동', '충무동', '봉암동', '현동', '자산동', '덕산동', '봉림동', '완월동', '동읍', '내서읍',
       '북면', '대산면', '구산면', '진북면', '진전면', '명곡동', '팔룡동', '의창동', '웅동2동',
       '양덕2동', '웅동1동', '웅남동', '반송동', '용지동', '회원2동', '월용동', '웅천동', '반월중앙동',
       '구암2동', '문화동'], dtype=object)

In [344]:
final.to_csv('07_29_병원전처리_규영.csv', encoding='cp949', index=False)

In [342]:
final.groupby('행정동')['요양기관명'].size().reset_index(name='요양기관수').sort_values(by='요양기관수', ascending=False)

,행정동,요양기관수
19,상남동,163
21,석전동,102
38,중앙동,61
25,오동동,61
43,팔룡동,57
34,의창동,56
6,내서읍,55
17,사파동,49
14,봉림동,42
10,명곡동,38
